<a href="https://colab.research.google.com/github/xiaoyangxuoo/Houston_crime_prevention/blob/master/Exploratory_analysis_of_HPD_crime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The first section, combining all data and read all data to dataframe

### Link for HPD crime data: https://drive.google.com/drive/folders/1p-cn9XjDSjoUPlPj0cBsgWNqbxlcGgK4?usp=sharing

In [ ]:
%%time 
# ########   Here is a link for geopandas usage: https://colab.research.google.com/github/shakasom/GDS/blob/master/Part1%20-%20Introduction.ipynb#scrollTo=6pMKuuyNMnwv
# # Important library for many geopython libraries
# !apt install gdal-bin python-gdal python3-gdal 
# # Install rtree - Geopandas requirment
# !apt install python3-rtree 
# # Install Geopandas
# !pip install git+git://github.com/geopandas/geopandas.git
# # Install descartes - Geopandas requirment
# !pip install descartes 
# # Install Folium for Geographic data visualization
# !pip install folium
# # Install plotlyExpress
# !pip install plotly_express


import pandas as pd
import numpy as np
import cv2
import os 
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import sys
print(sys.version)

In [ ]:
!pip install quickda

In [ ]:
!pip install pandas-profiling==2.8.0

In [ ]:
#data = os.path.join('/content/gdrive/My Drive/HPD crime statistics', 'HPD 2019.xlsx')
#df = pd.read_excel(data,header=0)

data = os.path.join('/content/gdrive/My Drive/GIT project', 'HPD 2019.xlsx')
df = pd.read_excel(data,header=0)
df.head()

In [ ]:
########## Call quickda package to do the EDA for me!
import os
from quickda.explore_data import *
from quickda.clean_data import *
from quickda.explore_numeric import *
from quickda.explore_categoric import *
from quickda.explore_numeric_categoric import *
from quickda.explore_time_series import *

In [ ]:
# explore(df, method="profile", report_name="Dataset Report", 
#         is_large_dataset=False)

In [ ]:
df.drop(columns=['Incident','Street\nType','Block Range','Suffix'],inplace=True)

In [ ]:
df = df.rename(columns={'Occurrence\nDate':'Date', 'Occurrence\nHour':'Hour', 'NIBRS\nClass':'Reporting_Class','NIBRSDescription':'Category','Offense\nCount':'Count','ZIP Code':'Zip_Code'})

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

## Convert descriptions to string

In [ ]:
####convert descriptions to string
df[["Zip_Code","Reporting_Class",'Category','Beat', 'Premise','StreetName','City']] = df[["Zip_Code","Reporting_Class",'Category','Beat', 'Premise','StreetName','City']].astype(str) 
##############truncate zip code
df['Zip_Code'] = df['Zip_Code'].apply(lambda x: x[:5])
df['Zip_Code_Numeric'] = df['Zip_Code'].astype(int)

In [ ]:
df.head()

## Finding outliers of each numerical column

In [ ]:
# import seaborn as sns
# ######Helper function using z_score to detect the outlier
# def detect_outlier(data_1):
#     outliers = []
#     threshold = 3
#     mean_1 = np.mean(data_1)
#     std_1 =np.std(data_1)    
    
#     for y in data_1:
#         z_score= (y - mean_1)/std_1 
#         if np.abs(z_score) > threshold:
#             outliers.append(y)
#     return outliers

### First column: Hour

In [ ]:
# sns.boxplot(x=df['Hour'])

In [ ]:
# detect_outlier(df['Hour'])

### Second column: Count

In [ ]:
# sns.boxplot(x=df['Count'])

In [ ]:
# detect_outlier(df['Count'])

In [ ]:
# df['Count'].unique()

In [ ]:
# df[df['Count']==65]

In [ ]:
# df[df['Count']==10]

In [ ]:
# df[df['Count']==1]

# Factors that may need to be considered

1) poverty : ##http://www.houstonstateofhealth.com/indicators/index/view?indicatorId=240&localeId=38555
  drug
  property
  aggravated assault, violent crimes
2) weather: 
3) average housing price (poverty moves to a different zip code
4） education level: http://www.houstonstateofhealth.com/?module=demographicdata&controller=index&action=index&id=38500&sectionId=938
5) political affiliation: 
6) utility usage
7) public transportation density


Therefore, it seems to be reasonable to group the count of each category and rank it from top to bottom

In [ ]:
# tempdf = df[['Category','Count']]
# tempdf = tempdf.groupby(['Category']).sum()
# Agg_cate = tempdf.reset_index()
# Agg_cate.sort_values(by='Count',ascending = False, inplace=True)
# Agg_cate = Agg_cate.reset_index(drop=True)
# Agg_cate.head(10)

It looks like our city has the most crime being reported as theft from motor vehicle and simple assault.

### Last column: Zip code

In [ ]:
#sns.boxplot(x=df['Zip_Code_Numeric'])

In [ ]:
# df['Zip_Code_Numeric'].unique()

It turns out that our dataset contains a lot of areas outside of Houston city limit

In [ ]:
Houston_df = df[df['Zip_Code'].str.startswith('77')] ##### Restrict our analysis within Houston city limit

In [ ]:
del df

In [ ]:
Houston_df.head()

The following plots can be done in quickDA

In [ ]:
# import matplotlib.pyplot as plt

# # An "interface" to matplotlib.axes.Axes.hist() method
# n, bins, patches = plt.hist(x = Houston_df['Hour'], bins=10, color='#0504aa',
#                             alpha=0.7, rwidth=0.85)
# plt.grid(axis='y', alpha=0.75)
# plt.xlabel('Hour')
# plt.ylabel('Frequency')
# plt.title('Histogram of crimes in Houston')
# maxfreq = n.max()
# # Set a clean upper y-axis limit.
# plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)

With no doubt, the safest time of Houston is between 2am to 7am

## Next, I would like to see if there is a monthly pattern in the crime dataset

### First, we need to extract the months

In [ ]:
Houston_df['Month'] = pd.DatetimeIndex(Houston_df['Date']).month

In [ ]:
Houston_df.head()

In [ ]:
# tempdf = Houston_df[['Month','Count']]
# tempdf = tempdf.groupby(['Month']).sum()
# Agg_mo = tempdf.reset_index()
# Agg_mo.sort_values(by='Count',ascending = False, inplace=True)
# Agg_mo = Agg_mo.reset_index(drop=True)
# Agg_mo.head(10)

It looks like the May and July is the most frequent crime dates which I think might be due to the weather? Not sure yet

## Then try to see the crime aggregation by premise

In [ ]:
# tempdf = Houston_df[['Premise','Count']]
# tempdf = tempdf.groupby(['Premise']).sum()
# Agg_premise = tempdf.reset_index()
# Agg_premise.sort_values(by='Count',ascending = False, inplace=True)
# Agg_premise = Agg_premise.reset_index(drop=True)
# Agg_premise.head(10)

## Lastly try to see the crime aggregation by zip_code

In [ ]:
# tempdf = Houston_df[['Zip_Code','Count']]
# tempdf = tempdf.groupby(['Zip_Code']).sum()
# Agg_zip = tempdf.reset_index()
# Agg_zip.sort_values(by='Count',ascending = False, inplace=True)
# Agg_zip = Agg_zip.reset_index(drop=True)
# Agg_zip.head(10)

It looks like the Chinatown of Houston suffers most crimes, and the University of Houston main campus neighbourhood also
Therefore, I will try to aggregate the zip_code, premise, and category together and see if it helps

In [ ]:
# tempdf = Houston_df[['Zip_Code','Premise','Category','Count']]
# tempdf = tempdf.groupby(['Zip_Code','Premise','Category']).sum()
# Agg_all = tempdf.reset_index()
# Agg_all

In [ ]:
#Agg_all.columns

Sort the counts within each group

In [ ]:
# All_zipcodes = Agg_all['Zip_Code'].unique()
# result = []
# for zip_code in All_zipcodes:
#   temp_zip = Agg_all[Agg_all['Zip_Code']== zip_code]
#   All_premises = temp_zip['Premise'].unique()
#   for prem in All_premises:
#     temp_prem = temp_zip[temp_zip['Premise']==prem]
#     temp_prem = temp_prem.sort_values(by='Count',ascending=False)
#     result.append(temp_prem)     
# Final = pd.concat(result)

In [ ]:
#Final

In [ ]:
Houston_df.head()

In [ ]:
!pip install catboost
!pip install -U scikit-learn

In [ ]:
# from sklearn.model_selection import train_test_split
# import catboost

In [ ]:
# train, validation = train_test_split(Houston_df, test_size=0.1)

In [ ]:
# def gen_pool(train, in_feature, out_feature):
#   train_pool = catboost.Pool(train[in_feature], train[out_feature], cat_features=train[in_feature].columns[train[in_feature].dtype=='object'])
#   return train_pool

## Collecting feature vectors

In [ ]:
from pathlib import Path
#directory = '/content/gdrive/My Drive/HPD crime statistics/Feature data'
directory = '/content/gdrive/My Drive/GIT project/Feature data'
feature_dfs = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    print(f)
    if f.endswith(".csv"):
      feature_dfs.append(pd.read_csv(f))

print(len(feature_dfs))
poverty_df = feature_dfs[0]
poverty_df.head()
poverty_df = poverty_df[poverty_df['Period of Measure'].str[-4:]=='2019']


In [ ]:
poverty_df.tail()

In [ ]:
final_df = Houston_df.merge(poverty_df[['Location', 'Indicator Rate Value']],
               left_on='Zip_Code_Numeric', right_on='Location', how='left')
final_df.head()
#Houston_df.drop(columns=['Location_x', 'Indicator Rate Value_x','Location_y'],inplace=True).rename(columns={'Indicator Rate Value_y': 'poverty rate'})

In [ ]:
final_df.shape

In [ ]:
# delete all the references to poverty table to release memory
del poverty_df
del feature_dfs[0]

In [ ]:
###### Filter out the 2019, the only relevant data columns
housing_df = feature_dfs[1]
filter_cols = ['RegionName', 'StateName', 'State', 'City']
temp = [col for col in housing_df.columns if col.startswith('2019')]
filter_cols.extend(temp)
housing_df_2019 = housing_df[filter_cols]
housing_df_2019 = housing_df_2019[(housing_df_2019['State']== 'TX') & (housing_df_2019['StateName'] == 'TX') & (housing_df_2019['City'] == 'Houston')]
housing_df_2019 = housing_df_2019.drop(['StateName', 'State', 'City'], axis=1)
housing_df_2019.head()

In [ ]:
final_df.tail()

In [ ]:
sum(housing_df_2019['RegionName'].isin([77489]))

In [ ]:
# melt housing_df data from wide format to long format
def melt(df, col_vals, key, value):
    keep_vals = df.columns.difference(col_vals)
    melted_sections = []
    for c in col_vals:
        melted_c = df[keep_vals].copy()
        melted_c[key] = c
        melted_c[value] = df[c]
        melted_sections.append(melted_c)
    return pd.concat(melted_sections)

In [ ]:
housing_melt = melt(housing_df_2019, ["2019-01-31", "2019-02-28", "2019-03-31", "2019-04-30", "2019-05-31", "2019-06-30", \
                                       "2019-07-31", "2019-08-31", "2019-09-30", "2019-10-31", "2019-11-30", "2019-12-31"], key='Time', value='Housing')

In [ ]:
housing_melt['Time'] = pd.to_datetime(housing_melt['Time'], format='%Y-%m-%d')
housing_melt['Month'] = housing_melt['Time'].dt.month

In [ ]:
# combine housing_melt with Houston_df_w_poverty
final_df = final_df.merge(housing_melt[['RegionName', 'Month', 'Housing']],
               left_on=['Month',"Zip_Code_Numeric"], right_on=['Month','RegionName'], how='left')
final_df.head()

In [ ]:
final_df[final_df['Zip_Code_Numeric'] == 77084].head()

In [ ]:
final_df.shape

In [ ]:
del housing_df_2019
del feature_dfs[1]

### The following code requires 13 min + to run, which is inefficient, but doable

In [ ]:
# Houston_df_w_poverty['Housing Price'] = 0.0
# #import math
# for idx, row in Houston_df_w_poverty.iterrows():
#   mon = Houston_df_w_poverty.loc[idx, 'Date']
#   mon = mon.strftime("%m")
#   #print(mon)
#   filter_cols = [col for col in housing_df_2019.columns if col[5:7] == mon]
#   filter_cols.append('RegionName')
#   temp = housing_df_2019[filter_cols]
#   # print(temp.head)
#   # break
#   if  temp[temp['RegionName']==Houston_df_w_poverty.loc[idx,'Zip_Code_Numeric']].values.size == 0:
#     continue
#   else:
#     Houston_df_w_poverty.loc[idx, 'Housing Price'] = temp[temp['RegionName']==Houston_df_w_poverty.loc[idx,'Zip_Code_Numeric']].iloc[0, 0]

In [ ]:
weather_df = feature_dfs[0]
weather_df.head()

In [ ]:
weather_df = weather_df[['Date time', 'Temperature', 'Conditions']]
weather_df.head()

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
weather_df['time'] = pd.to_datetime(weather_df['Date time'], format='%m/%d/%Y %H:%M:%S')
weather_df['Hour'] = weather_df['time'].dt.hour
weather_df['Date'] = weather_df['time'].dt.date
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
weather_df = weather_df[weather_df['time'].dt.year == 2019]
weather_df = weather_df.drop(['Date time','time'], axis = 1)
weather_df.head()

In [ ]:
# delete duplicates in weather data
weather_df = weather_df.drop_duplicates(subset=['Hour', 'Date'], keep='first')

In [ ]:
# merge weather data according to date and hour
final_df = final_df.merge(weather_df[['Temperature', 'Conditions', 'Hour', 'Date']],
               left_on=['Date','Hour'], right_on=['Date','Hour'], how='left')
final_df.rename(columns = {'Indicator Rate Value': 'Poverty_rate', 'Conditions':
                                               'Weather_conditions'}, inplace = True)
final_df.head()

In [ ]:
final_df.shape

In [ ]:
del weather_df
del feature_dfs[0]

In [ ]:
unemploy_df = feature_dfs[0]
unemploy_df.head()

In [ ]:
unemploy_df = unemploy_df[unemploy_df['Year'] == 2019]
m = {'M01':1, 'M02':2, 'M03':3, 'M04':4, 'M05':5, 'M06':6, 'M07':7, 'M08':8, 'M09':9, 'M10':10, 'M11':11, 'M12':12}
unemploy_df['Month'] = unemploy_df.Period.map(m)
unemploy_df = unemploy_df.drop(['Series ID','Period','Label'], axis = 1)
unemploy_df.head()

In [ ]:
final_df['Year'] = final_df.Date.dt.year

In [ ]:
# merge unemployment data according to year and month
final_df = final_df.merge(unemploy_df[['Year', 'Value', 'Month']],
               left_on=['Year','Month'], right_on=['Year','Month'], how='left')
final_df.rename(columns = {'Value': 'Unemploy_value'}, inplace = True)
final_df.head()

In [ ]:
final_df.shape

In [ ]:
del unemploy_df
del feature_dfs[0]

In [ ]:
income_df = feature_dfs[0]
income_df.head()

In [ ]:
income_df = income_df[income_df['Period of Measure'] == '2015-2019']

In [ ]:
# merge Median Household Income data according to zipcode
final_df = final_df.merge(income_df[['Location', 'Indicator Rate Value']],
               left_on=['Zip_Code_Numeric'], right_on=['Location'], how='left')
final_df.rename(columns = {'Indicator Rate Value': 'Median_household_income'}, inplace = True)
final_df.head()

In [ ]:
final_df.shape

In [ ]:
del income_df
del feature_dfs[0]

In [ ]:
# bachelor_df = feature_dfs[0]
# bachelor_df.head()

In [ ]:
# # merge 25+ with Bachelor or higher data according to zipcode
# final_df = final_df.merge(bachelor_df[['Location', 'Indicator Rate Value']],
#                left_on=['Zip_Code_Numeric'], right_on=['Location'], how='left')
# final_df.rename(columns = {'Indicator Rate Value': '25+_w_bachelor_or_higher_in_percent'}, inplace = True)
# final_df.head()

In [ ]:
# del bachelor_df
# del feature_dfs[0]

In [ ]:
high_school_df = feature_dfs[1]
high_school_df.head() 

In [ ]:
high_school_df = high_school_df[high_school_df['Period of Measure'] == '2015-2019']

In [ ]:
# merge 25+ with high school or higher data according to zipcode
final_df = final_df.merge(high_school_df[['Location', 'Indicator Rate Value']],
               left_on=['Zip_Code_Numeric'], right_on=['Location'], how='left')
final_df.rename(columns = {'Indicator Rate Value': '25+_w_high_school_or_higher_in_percent'}, inplace = True)
final_df.head()

In [ ]:
del high_school_df
del feature_dfs[1]

In [ ]:
Houston_df.shape

In [ ]:
final_df.shape

In [ ]:
from google.colab import files
final_df.to_csv('Final.csv') 
files.download('Final.csv')

In [ ]:
# # merge per_capita_income data according to zipcode
# final_df = final_df.merge(per_capita_income[['Location', 'Indicator Rate Value']],
#                left_on=['Zip_Code_Numeric'], right_on=['Location'], how='left')
# final_df.rename(columns = {'Indicator Rate Value': 'Per_capita_income'}, inplace = True)
# final_df.head()

In [ ]:
# del per_capita_income
# del feature_dfs[0]